In [1]:
import torch
import torch.nn as nn
import tiktoken


In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers (# of transformer blocks)
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))
    
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            # Hidden layer neurons = 4 * len(emb dim)
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            # Non linear activation function
            GELU(),
            # Must squish output back down to emb dim so that each transformer block will output same length tensor
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [4]:
# Attention class
class MultiHeadAttention(nn.Module):
    #Constructor Inputs:d in is the embedding vector length, d out is the context vector length, context length is 
    # the number of tokens, dropout is the rate of data points left out, num_heads is the number of attention heads you want running
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
# forward is called whenever class is called on a certain bacthed data (x is the batched data)
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [5]:
class GPTModel(nn.Module):
    def __init__(self, cfg):    # cfg refers to the gpt config dictionary above
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]) # makes/unpacks a list of transformer blocks, stuck together as modules
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        # Makes embedding vectors from input ids
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        # Transformer stuff
        x = self.drop_emb(x)
        # all the transformer blocks
        x = self.trf_blocks(x)
        #Final normalization step
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits   # tensor of scores like probabilities: logits = log of the odds

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.att = MultiHeadAttention(
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"]
            )
        self.dropout = nn.Dropout(cfg["drop_rate"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.feedforward = FeedForward(cfg)

    def forward(self, x):
        #Attention chunk of the transformer:

        #store original x as variable to add back later in shortcut step
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        s = self.dropout(x)
        x = x + shortcut

        #Feed forward chunk of the transformer:

        shortcut = x
        x = self.norm2(x)
        x = self.feedforward(x)
        x = self.dropout(x)
        s = x + shortcut

        return x

# Keeps the numbers from getting wild
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim)) # make scale and shift trainable
        self.shift = nn.Parameter(torch.zeros(emb_dim)) # Start as 1 and 0, so they may learn to undo the normalization?

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        #Computing z scores
        norm_x = (x - mean) / torch.sqrt(var + self.eps) # eps helps avoid dividing by super small numbers (unstable)
        #self.scale and self.shift are learnable: model can learn to undo normalization
        return self.scale * norm_x + self.shift 
        

In [6]:
# just an example of unpacking or * function
primes = [2, 3, 5, 7, 11]
print(primes)
print(*primes)

[2, 3, 5, 7, 11]
2 3 5 7 11


In [7]:
tokenizer = tiktoken.get_encoding("gpt2")

In [8]:
batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [9]:
model = GPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)
#outputs probabilities of the next word being that word in the vocab (highest score in the vector is the most probably next word in the vocab)
# Each vector represents the chance that the next word is that positions word in the vocab

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.4808,  0.3746, -1.0565,  ..., -0.0827,  0.6796, -0.5813],
         [ 0.7857, -0.2391, -0.5553,  ...,  0.1997,  1.1375, -0.8066],
         [ 0.4304, -0.3994, -0.9635,  ...,  0.4287,  0.8149, -0.5429],
         [ 0.4249,  0.0866, -0.7264,  ...,  0.6482,  1.1453, -0.6743]],

        [[ 1.2019,  0.0487,  0.0824,  ...,  0.0858,  1.0866, -0.6949],
         [ 1.6246, -0.1845, -0.0800,  ...,  0.0123,  1.5118, -1.0835],
         [ 1.3717, -0.2173, -0.2796,  ...,  0.3951,  1.4553, -1.3637],
         [ 1.9682, -0.5965, -0.0604,  ..., -0.0864,  1.7573, -0.9559]]],
       grad_fn=<UnsafeViewBackward0>)


In [10]:
txt1

'Every effort moves you'

In [11]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to find probabilities from logits
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

## Example of it Maybe Working

In [12]:
# Create a string
start_context = "Hello, I am"

# Turns str of text into list of token IDs
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

# Turns list of IDs into a tensor that looks like batch (unsqueeze)
# Unsqueeze adds a dimension []
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [13]:
# List
print(encoded)

# Batch-like Tensor
print(encoded_tensor)

[15496, 11, 314, 716]
tensor([[15496,    11,   314,   716]])


In [14]:
# Unsqueeze example
print(torch.tensor(encoded).shape)
print(torch.tensor(encoded).unsqueeze(0).shape)

torch.Size([4])
torch.Size([1, 4])


In [15]:
out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 49319, 20011, 30240, 14093, 22509,  6514]])
Output length: 10


In [16]:
#Turns IDs back into a list
out.squeeze(0).tolist()

[15496, 11, 314, 716, 49319, 20011, 30240, 14093, 22509, 6514]

In [18]:
tokenizer.decode(out.squeeze(0).tolist())

'Hello, I am pim Compl encoded brush Rhod wal'

In [20]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [21]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

In [22]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


Output text:
 Every effort moves you Ensurepasteiologicalopping earlier AngusSocket NEOBeckulk


In [24]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]


In [25]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)


torch.Size([2, 3, 50257])


In [26]:
probas

tensor([[[7.3477e-05, 3.3218e-05, 1.2232e-05,  ..., 1.1049e-05,
          8.1190e-05, 7.4210e-06],
         [4.2083e-05, 2.9924e-05, 9.0092e-06,  ..., 1.1778e-05,
          6.4809e-05, 6.2278e-06],
         [5.6314e-05, 2.7321e-05, 1.5721e-05,  ..., 1.1442e-05,
          3.9843e-05, 1.4492e-05]],

        [[3.7620e-05, 2.0735e-05, 2.2633e-05,  ..., 1.5965e-05,
          3.1063e-05, 6.8390e-06],
         [4.5913e-05, 2.3239e-05, 8.7363e-06,  ..., 1.6845e-05,
          4.8912e-05, 1.0129e-05],
         [5.0346e-05, 1.3835e-05, 1.4882e-05,  ..., 1.4814e-05,
          3.7552e-05, 9.7041e-06]]])

In [ ]:
# Predicted tokens: (obviously very wrong)
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[16019],
         [29256],
         [49248]],

        [[30274],
         [30274],
         [ 3765]]])


In [28]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  solved permittingaband


In [ ]:
text_idx = 0
# Gives us probabilities in the spots that we care abt (the target word in the vocab)
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)


Text 1: tensor([1.2203e-05, 1.7063e-05, 1.9621e-05])
Text 2: tensor([9.6240e-06, 1.6365e-05, 1.9042e-05])


In [ ]:
# Compute logarithm of all token probabilities
# We want these numbers close to zero
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([-11.3138, -10.9786, -10.8389, -11.5513, -11.0204, -10.8689])


In [ ]:
# Score we use to do assessment (Cross Entropy)
# Calculate the average probability for each token
# We want this to be close to zero
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(-11.0953)
tensor(11.0953)
